# Theoretical Modelling of Composite Galaxies 
This code is intended to output a dataframe containing the colours of the theoretical galaxy+AGN composites.



In [142]:
# Read in an AGN template
# Read in all required libraries
# Import in all of the required libraries
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np
import pandas as pd
import os
from astLib import astSED
import astropy.io.fits as fits
import matplotlib.path as mpath
from carf import * # custom module for functions relating to the project

# So that we can change the helper functions without reloading the kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [143]:
# In addition also use astSED to create filters

# Read in all filters
# UVJ Filters
pb_U_path = os.path.join('datasets', 'Filters', 'Generic_Johnson.U.dat')
pb_V_path = os.path.join('datasets', 'Filters', 'Generic_Johnson.V.dat')
pb_J_path = os.path.join('datasets', 'Filters', '2MASS_2MASS.J.dat')
# Spitzer filters
pb_f3_6_path = os.path.join('datasets', 'filters', 'Spitzer_IRAC.I1.dat')
pb_f4_5_path = os.path.join('datasets', 'filters', 'Spitzer_IRAC.I2.dat')
pb_f5_8_path = os.path.join('datasets', 'filters', 'Spitzer_IRAC.I3.dat')
pb_f8_0_path = os.path.join('datasets', 'filters', 'Spitzer_IRAC.I4.dat')
# ugr filters
pb_u_path = os.path.join('datasets', 'filters', 'Paranal_OmegaCAM.u_SDSS.dat')
pb_g_path = os.path.join('datasets', 'filters', 'Paranal_OmegaCAM.g_SDSS.dat')
pb_r_path = os.path.join('datasets', 'filters', 'Paranal_OmegaCAM.r_SDSS.dat')


# Load all of the filters 
pb_U = astSED.Passband(pb_U_path, normalise=False)
pb_V = astSED.Passband(pb_V_path, normalise=False)
pb_J = astSED.Passband(pb_J_path, normalise=False)
pb_f3_6 = astSED.Passband(pb_f3_6_path, normalise=False)
pb_f4_5 = astSED.Passband(pb_f4_5_path, normalise=False)
pb_f5_8 = astSED.Passband(pb_f5_8_path, normalise=False)
pb_f8_0 = astSED.Passband(pb_f8_0_path, normalise=False)
pb_u = astSED.Passband(pb_u_path, normalise=False)
pb_g = astSED.Passband(pb_g_path, normalise=False)
pb_r = astSED.Passband(pb_r_path, normalise=False)

filter_set = {'U': pb_U, 'V':pb_V, 'J':pb_J, 'IRAC3.6': pb_f3_6,'IRAC4.5': pb_f4_5,'IRAC5.8': pb_f5_8, 'IRAC8.0':pb_f8_0, 'u': pb_u, 'g': pb_g, 'r':pb_r}



In [144]:
# Skirtor models
skirtor_folderpath = os.path.join('datasets', 'Templates', 'Skirtor')

# Swire templates
swire_folderpath = os.path.join('datasets', 'Templates', 'SWIRE')

# Brown templates
brown_folderpath = os.path.join('datasets', 'Templates', 'Brown', '2014','Rest')


In [145]:
# Read in the AGN templates
type1_agn, type1_params = create_type1_skirtor_agn(skirtor_folderpath)
type2_agn, type2_params = create_type2_skirtor_agn(skirtor_folderpath)

# Read in the template set of choice
swire_templates, template_names = read_swire_templates(swire_folderpath)

# Read in the brown templates
brown_templates, brown_template_names = read_brown_galaxy_templates(brown_folderpath)

In [146]:
# Choose running parameters

# Template set and AGN type
agn_model_name = 'Type1AGN'
template_set_name = 'Brown'

# Setup how many alpha values we want to explore
alpha_values = np.linspace(0, 1, 11)

In [147]:
# Choose an AGN model
if agn_model_name == 'Type1AGN':
    agn_model = type1_agn
elif agn_model_name == 'Type2AGN':
    agn_model = type2_agn
else:
    print('AGN model not recognised')

if template_set_name == 'SWIRE':
    template_set = swire_templates
    template_names = template_names
elif template_set_name == 'Brown':
    template_set = brown_templates
    template_names = brown_template_names
else:
    print('Template set not recognised')

In [148]:
# Check that all of the column names are easily able to be viewed
composite_fluxes

,id,z,U_0,U_10,U_20,U_30,U_40,U_50,U_60,U_70,...,r_10,r_20,r_30,r_40,r_50,r_60,r_70,r_80,r_90,r_100
0,Arp_118,0.0,-21.198908,-21.315776,-21.421279,-21.517432,-21.605757,-21.687435,-21.763396,-21.834388,...,-23.454779,-23.475153,-23.495151,-23.514788,-23.534076,-23.553028,-23.571654,-23.589966,-23.607974,-23.625689
1,Arp_118,0.1,-20.237172,-20.287567,-20.335726,-20.381839,-20.426074,-20.468576,-20.509478,-20.548894,...,-21.492481,-21.513877,-21.534859,-21.555444,-21.575645,-21.595478,-21.614955,-21.634088,-21.65289,-21.671372
2,Arp_118,0.2,-20.201401,-20.302567,-20.395105,-20.480372,-20.559427,-20.633115,-20.702117,-20.766996,...,-21.460665,-21.504041,-21.545751,-21.585917,-21.624651,-21.66205,-21.698203,-21.733192,-21.767088,-21.799958
3,Arp_118,0.3,-17.401194,-17.657027,-17.863926,-18.037638,-18.187352,-18.318902,-18.436221,-18.542091,...,-19.901468,-19.939198,-19.975661,-20.010938,-20.045106,-20.078231,-20.110376,-20.141596,-20.171943,-20.201465
4,Arp_118,0.4,-20.094718,-20.244207,-20.375583,-20.492764,-20.598521,-20.694885,-20.783389,-20.865219,...,-21.546386,-21.601016,-21.653028,-21.702663,-21.750127,-21.795603,-21.83925,-21.881211,-21.921609,-21.960559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,UM_461,0.3,24.632331,23.90264,23.470149,23.161691,22.921756,22.725366,22.559117,22.414976,...,22.184383,22.046727,21.924576,21.814789,21.71509,21.623782,21.53956,21.461404,21.388498,21.320181
1020,UM_461,0.4,23.150962,22.830091,22.582733,22.381405,22.211634,22.064854,21.935575,21.820064,...,21.246951,21.163347,21.085722,21.013279,20.945369,20.881457,20.8211,20.763921,20.709604,20.657876
1021,UM_461,0.5,23.503862,23.349919,23.215117,23.095218,22.987253,22.88906,22.799015,22.71587,...,21.826568,21.780132,21.735601,21.692825,21.65167,21.612019,21.573765,21.536813,21.501077,21.46648
1022,UM_461,0.6,26.405769,25.136712,24.567461,24.195932,23.919636,23.699581,23.516697,23.360224,...,23.121267,22.974438,22.845121,22.72958,22.62516,22.529908,22.442342,22.361315,22.285917,22.215416


In [149]:
# Create all of the composites
composites = generate_composite_set(agn_model, template_set, alpha_values)

In [221]:
# # # Save all the sampled colours to a data frame

# Create an empty data frame for each template that has the filters with the alpha values o.e u_0, u_10, u_20 etc for each filter
column_names = ['id', 'z'] # add an inital column for the redshift and for the ID
# filters 
for filter in filter_set.keys():
    # Check filter we are looking at
    print(filter)
    
    for alpha_val in alpha_values:
    # Add filter into a data frame
        column_names.append(filter + '_' + str(int(round(alpha_val, 2)*100)))

# Turn into a dataframe  
composite_fluxes = pd.DataFrame(columns=column_names)

# redshif
redshift_cutoff = 4

# Try a more simple approach to this
# I have a set of composites
filter_sets = []

redshifts = []
ids = []

# Create a list of empty lists for each filter
for filter in filter_set.keys():
    filter_sets.append([])
    
    
# Add the redshifts and ids to the data frame
for j in range(len(template_names)):
    for redshift in np.arange(0, redshift_cutoff, 0.1):
        redshifts.append(redshift)
        ids.append(template_names[j])
    


U
V
J
IRAC3.6
IRAC4.5
IRAC5.8
IRAC8.0
u
g
r


In [222]:
# Add to the df
composite_fluxes['id'] = ids
composite_fluxes['z'] = redshifts

In [223]:
composite_fluxes

,id,z,U_0,U_10,U_20,U_30,U_40,U_50,U_60,U_70,...,r_10,r_20,r_30,r_40,r_50,r_60,r_70,r_80,r_90,r_100
0,Arp_118,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arp_118,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arp_118,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arp_118,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arp_118,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5115,UM_461,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5116,UM_461,3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5117,UM_461,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5118,UM_461,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# An easier way to do it would be like so

# Loop through the id's is number 1
    # Loop through the redshifts is number 2
        # Loop through the filters is number 3
            # Loop through the alpha values is number 4
                # Add the flux to the composite_fluxes

# # Turn the above into code
# for i in range(len(template_names)):
#     for j in np.arange(0, redshift_cutoff, 0.1):
#         for filter in filter_set.keys():
#             for alpha_val in alpha_values:
#                 # Get the composite
#                 composite = composites[(template_names[i], alpha_val)]
#                 # Add the flux to the composite_fluxes
#                 composite_fluxes[filter + '_' + str(int(round(alpha_val, 2)*100))][(composite_fluxes['id'] == template_names[i]) & (composite_fluxes['z'] == j)] = flux



In [217]:
# Create a list that will have an empty list for each filter at each alpha value
filter_sets = []
for filter in filter_set.keys():
    for alpha_val in alpha_values:
        filter_sets.append([])
        
#filter_sets[0].append([]) # Add an empty list for the redshifts

# print the unique filters which is every heading in the dataframe except for the id and z
print(composite_fluxes.columns)
unique_filters = composite_fluxes.columns[2:]
# 
#print(filter_set.keys())

Index(['id', 'z', 'U_0', 'U_10', 'U_20', 'U_30', 'U_40', 'U_50', 'U_60',
       'U_70',
       ...
       'r_10', 'r_20', 'r_30', 'r_40', 'r_50', 'r_60', 'r_70', 'r_80', 'r_90',
       'r_100'],
      dtype='object', length=112)


In [219]:
composite_fluxes

,id,z,U_0,U_10,U_20,U_30,U_40,U_50,U_60,U_70,...,r_10,r_20,r_30,r_40,r_50,r_60,r_70,r_80,r_90,r_100
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [209]:
filter_sets

# U will be at 2 to 12
U_start = 2

# V will be at 13 to 23
V_start = 13

# J will be at 24 to 34
J_start = 24

# IRAC3.6 will be at 35 to 45
IRAC3_6_start = 35

# IRAC4.5 will be at 46 to 56
IRAC4_5_start = 46

# IRAC5.8 will be at 57 to 67
IRAC5_8_start = 57

# IRAC8.0 will be at 68 to 78
IRAC8_0_start = 68

# u will be at 79 to 89
u_start = 79
# g will be at 90 to 100
g_start = 90

# r will be at 101 to 111
r_start = 101

# This means we can find the _10 values by adding 1 one to the index of the _0 values


print(composite_fluxes.columns[111])

r_100


In [210]:
filter_sets

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [211]:

# for i in range(len(alpha_values)):
#     # This will be the set of composites for the specific alpha value
#     sed_alpha_data = composites[i]
    
#     for j, sed_data in enumerate(sed_alpha_data):
#         # Create an SED object using astSED
    
#         #print(sed_data['lambda (Angstroms)'])
#         wl = sed_data['lambda (Angstroms)']
#         fl = sed_data['Total Flux (erg/s/cm^2/Angstrom)']
        
#         # Save the sed, and export it to a file, with an associated alpha value appended to the name

        
#         # Only turn this on to output all seds
#         sed_data.to_csv(f'outputs/composite_seds/{template_set_name}/'+template_names[j]+ f'{agn_model_name}'+ 'composite_' +str(int(round(alpha_values[i], 2)*100))+'.csv')
        
#         # Create an SED object using astSED
#         sed = astSED.SED(wavelength=wl, flux=fl, z=0.0) # Fresh SED with a particular wavelength and flux value
        
        
        
#         # Now we need to redshift the SED templates to the appropriate redshifts, we are going from 0 to 4 in steps of 0.1
#         for z in np.arange(0, redshift_cutoff, 0.1): #  Redshift the SED
            
#             # We can now redshift the SED to the approriate redshift
#             sed.redshift(z)
            
     
#             # Calculate each filter value for the specific alpha value -  additionally also add the redshift to the data frame
#             for filter in filter_set.keys(): 
                
#                 # Calculate the magnitude for the filter - at the specific redshift
#                 mag = astSED.SED.calcMag(sed, filter_set[filter], magType='AB')
                
#                 # Map the filter to the correct index
#                 if filter == 'U':
#                     index = U_start
#                 elif filter == 'V':
#                     index = V_start
#                 elif filter == 'J':
#                     index = J_start
#                 elif filter == 'IRAC3.6':
#                     index = IRAC3_6_start
#                 elif filter == 'IRAC4.5':
#                     index = IRAC4_5_start
#                 elif filter == 'IRAC5.8':
#                     index = IRAC5_8_start
#                 elif filter == 'IRAC8.0':
#                     index = IRAC8_0_start
#                 elif filter == 'u':
#                     index = u_start
#                 elif filter == 'g':
#                     index = g_start
#                 elif filter == 'r':
#                     index = r_start
                
                
#                 # put the mag into the filter set
#                 filter_sets[index + i].append(mag)
                
                
                
#                 # the mag is now associated with a filter, a redshift and an alpha value for a particular SED
        
#                 # row_index = (redshifts.index(z) * len(template_names)) + j
#                 # composite_fluxes.at[row_index, filter + '_' + str(int(round(alpha_values[i], 2) * 100))] = mag
                
                
                
                
                
                
                
#                 #  # Create a new row as a dictionary
#                 # new_row = {
#                 #     'id': template_names[j],
#                 #     'z': z,
#                 #     filter + '_' + str(int(round(alpha_values[i], 2) * 100)): mag
#                 # }
                
                
#                 # Add the new row to the data frame
#                 #composite_fluxes = composite_fluxes.append(new_row, ignore_index=True)
                
                
            
            
            
            

IndexError: list index out of range

In [212]:
len(redshifts)
len(ids)
#len(filter_sets*len(filter_sets[0]))


5120

In [213]:
# add each of these to the data frame
composite_fluxes['id'] = ids
composite_fluxes['z'] = redshifts

for i in range(len(filter_sets)):
    print(i)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109


In [214]:
composite_fluxes

,id,z,U_0,U_10,U_20,U_30,U_40,U_50,U_60,U_70,...,r_10,r_20,r_30,r_40,r_50,r_60,r_70,r_80,r_90,r_100
0,Arp_118,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arp_118,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arp_118,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arp_118,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arp_118,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5115,UM_461,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5116,UM_461,3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5117,UM_461,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5118,UM_461,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
# Output the CSV
#composite_fluxes.to_csv(f'outputs/composite_seds/{template_set_name}_theoretical_composite_fluxes_{agn_model_name}.csv')

In [132]:
# Ensure that the data is consistentcomposite_fluxes

In [133]:
# from concurrent.futures import ThreadPoolExecutor, as_completed

# # ... (Assuming composites, template_names, agn_model_name, filter_set are defined)

# # Setup how many alpha values we want to explore
# alpha_values = np.linspace(0, 1, 11)

# # Create column names (including filter_alpha combinations)
# column_names = ['id', 'z'] + [
#     f"{filter}_{int(round(alpha_val, 2) * 100)}"
#     for filter in filter_set.keys()
#     for alpha_val in alpha_values
# ]

# # Calculate the expected number of rows (unique id-z combinations)
# num_redshifts = len(np.arange(0, 4, 0.1))
# num_rows = len(composites) * len(composites[0]) * num_redshifts

# # Pre-allocate the DataFrame
# composite_fluxes = pd.DataFrame(index=np.arange(num_rows), columns=column_names)

# # Function to process each sed_data in parallel
# def process_sed_data(sed_data, i, j):
#     wl = sed_data['lambda (Angstroms)']
#     fl = sed_data['Total Flux (erg/s/cm^2/Angstrom)']

#     # ... (code for saving SED to file, if needed)

#     sed = astSED.SED(wavelength=wl, flux=fl, z=0.0)

#     # Calculate magnitudes for all filters and redshifts
#     redshifts = np.arange(0, 0.1, 0.1)
#     mags = np.zeros((len(redshifts), len(filter_set))) 
#     for k, z in enumerate(redshifts):
#         sed.redshift(z)
#         for l, filter in enumerate(filter_set.keys()):
#             mags[k, l] = astSED.SED.calcMag(sed, filter_set[filter], magType='AB')

#     # Create a dictionary to store results for this sed_data
#     result_dict = {}
#     for k, z in enumerate(redshifts):
#         # Key for this row in the DataFrame
#         row_key = (template_names[j], z)

#         # If the row doesn't exist yet, create it
#         if row_key not in result_dict:
#             result_dict[row_key] = {'id': template_names[j], 'z': z}

#         # Add magnitudes for all filters for this alpha value
#         for l, filter in enumerate(filter_set.keys()):
#             col_name = filter + '_' + str(int(round(alpha_values[i], 2) * 100))
#             result_dict[row_key][col_name] = mags[k, l]

#     return result_dict

# # Parallelize the processing of sed_data
# with ThreadPoolExecutor() as executor:
#     futures = []
#     for i in range(len(alpha_values)):
#         sed_alpha_data = composites[i]
#         for j, sed_data in enumerate(sed_alpha_data):
#             futures.append(executor.submit(process_sed_data, sed_data, i, j))

#     # Collect results from all threads and update the DataFrame
#     for future in as_completed(futures):
#         result_dict = future.result()
#         for row_key, row_data in result_dict.items():
#             composite_fluxes.loc[composite_fluxes['id'] == row_key[0], row_data.keys()] = row_data.values() 


,id,z,U_0,U_10,U_20,U_30,U_40,U_50,U_60,U_70,...,r_10,r_20,r_30,r_40,r_50,r_60,r_70,r_80,r_90,r_100
0,Arp_118,0.0,-21.198908,-21.315776,-21.421279,-21.517432,-21.605757,-21.687435,-21.763396,-21.834388,...,-23.454779,-23.475153,-23.495151,-23.514788,-23.534076,-23.553028,-23.571654,-23.589966,-23.607974,-23.625689
1,Arp_118,0.1,-20.237172,-20.287567,-20.335726,-20.381839,-20.426074,-20.468576,-20.509478,-20.548894,...,-21.492481,-21.513877,-21.534859,-21.555444,-21.575645,-21.595478,-21.614955,-21.634088,-21.65289,-21.671372
2,Arp_118,0.2,-20.201401,-20.302567,-20.395105,-20.480372,-20.559427,-20.633115,-20.702117,-20.766996,...,-21.460665,-21.504041,-21.545751,-21.585917,-21.624651,-21.66205,-21.698203,-21.733192,-21.767088,-21.799958
3,Arp_118,0.3,-17.401194,-17.657027,-17.863926,-18.037638,-18.187352,-18.318902,-18.436221,-18.542091,...,-19.901468,-19.939198,-19.975661,-20.010938,-20.045106,-20.078231,-20.110376,-20.141596,-20.171943,-20.201465
4,Arp_118,0.4,-20.094718,-20.244207,-20.375583,-20.492764,-20.598521,-20.694885,-20.783389,-20.865219,...,-21.546386,-21.601016,-21.653028,-21.702663,-21.750127,-21.795603,-21.83925,-21.881211,-21.921609,-21.960559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,UM_461,0.3,24.632331,23.90264,23.470149,23.161691,22.921756,22.725366,22.559117,22.414976,...,22.184383,22.046727,21.924576,21.814789,21.71509,21.623782,21.53956,21.461404,21.388498,21.320181
1020,UM_461,0.4,23.150962,22.830091,22.582733,22.381405,22.211634,22.064854,21.935575,21.820064,...,21.246951,21.163347,21.085722,21.013279,20.945369,20.881457,20.8211,20.763921,20.709604,20.657876
1021,UM_461,0.5,23.503862,23.349919,23.215117,23.095218,22.987253,22.88906,22.799015,22.71587,...,21.826568,21.780132,21.735601,21.692825,21.65167,21.612019,21.573765,21.536813,21.501077,21.46648
1022,UM_461,0.6,26.405769,25.136712,24.567461,24.195932,23.919636,23.699581,23.516697,23.360224,...,23.121267,22.974438,22.845121,22.72958,22.62516,22.529908,22.442342,22.361315,22.285917,22.215416
